# Assignment 2

## Roll number: 2023121013, 2023122002, 2023121006


### Instructions
 * Fill in the roll-number in the cell above.
 * Code must be submitted in Python in jupyter notebooks. We highly recommend using anaconda/miniconda distribution or at the minimum, virtual environments for this assignment.
 * All the code and result files should be uploaded in the github classroom.
 *  Most of the questions require you to **code your own functions** unless there is a need to call in the abilities of the mentioned libraries, such as Visualisation from Open3D. Make sure your code is modular since you will be reusing them for future assignments. All the functions related to transformation matrices, quaternions, and 3D projection are expected to be coded by you.
 *  All the representations are expected to be in a right-hand coordinate system.
<!--  * Answer to the descriptive questions should be answered in your own words. Copy-paste answers will lead to penalty. -->
 * You could split the Jupyter Notebook cells where TODO is written, but please try to avoid splitting/changing the structure of other cells.
 * All the visualization should be done inside the notebook unless specified otherwise.
 * Plagiarism will lead to heavy penalty.
 * Commit the notebooks in the repo and any other results files under the result folder in the GitHub Classroom repo. 
 * This is a group assignment. Discussions are encouraged but any sharing of code among different teams will be penalized. 

## SECTION 1: ICP with SVD

### 1.1 Perform Procrustes alignment on two point clouds with (given) known correspondences. (5 Points)

Let X be your point cloud observed from the initial pose P1. You then transform it to a new pose P2. Now you wish to apply ICP to recover transformation between (X & P1) and (X & P2).

Use toothless.ply point cloud and perform the alignment between the two point clouds using procrustes alignment. Your task is to write a function that takes two point clouds as input wherein the corresponding points between the two point clouds are located at the same index and returns the transformation matrix between them. Compute the alignment error after aligning the two point clouds.

<b>Use root mean squared error (RSME) as the alignment error metric.</b>

Make sure your code is modular as we will use this function in the next sub-part.

We will again use our own getTransform function to generate the 4x4 Transformation matrix to transform the pointcloud to P2, so make sure your code works for any general Transformation matrix

In [1]:
# Imports

import numpy as np
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def getTransform():
    """ Generates a random 4x4 transformation matrix. """

    R = o3d.geometry.get_rotation_matrix_from_xyz(np.random.rand(3) * 2 * np.pi)
    T = np.random.rand(3) * 5000
    transformation = np.vstack((np.hstack((R, T.reshape(-1, 1))), [0, 0, 0, 1]))
    return transformation

In [3]:
def apply_transformation(points, transformation):
    """ Applies a 4x4 transformation matrix to an array of points. """

    rotation = transformation[:3, :3]
    translation = transformation[:3, 3]
    transformed_points = (points @ rotation.T) + translation
    return transformed_points

In [4]:
def icp_known_correspondences(T):
    """ Given a transformation matrix, transforms toothless.ply
    and recovers the transformation using Procrustes alignment."""

    # Read the point cloud
    pcd = o3d.io.read_point_cloud("data/toothless.ply")
    X_P1 = np.asarray(pcd.points)

    # Apply the transformation
    X_P2 = apply_transformation(X_P1, T)

    # Predict the applied transformation
    T_pred = procrustes_alignment(X_P1, X_P2)

    # Apply the predicted transformation
    X_P2_pred = apply_transformation(X_P1, T_pred)

    # Compute RMSE between the two transformed point clouds
    error = np.sqrt(np.mean((X_P2 - X_P2_pred) ** 2))

    return T_pred, error

def procrustes_alignment(P, Q):
    """ Performs Procrustes alignment on two point clouds
    and returns the transformation matrix."""

    # Compute the mean of the point clouds
    P_mean = np.mean(P, axis=0)
    Q_mean = np.mean(Q, axis=0)

    # Center the two point clouds around their means
    P_centered = P - P_mean
    Q_centered = Q - Q_mean

    # Compute the cross-covariance matrix
    W = Q_centered.T @ P_centered

    # Compute the SVD of the cross-covariance matrix
    U, S, V_t = np.linalg.svd(W)

    # Compute the rotation and translation
    R = U @ V_t
    T = Q_mean - (R @ P_mean)

    # Construct the transformation matrix
    transformation = np.vstack((np.hstack((R, T.reshape(-1, 1))), [0, 0, 0, 1]))

    return transformation

T = getTransform()
T_pred, error = icp_known_correspondences(T)

print('Original Transformation:\n', T)
print('Predicted Transformation:\n', T_pred)
print('Alignment Error:', error)

Original Transformation:
 [[-5.76600617e-01  7.42965817e-01  3.39902227e-01  3.21665218e+03]
 [ 7.60254369e-01  3.35528968e-01  5.56267566e-01  1.20546238e+03]
 [ 2.99240743e-01  5.79156375e-01 -7.58309219e-01  4.33474833e+03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Predicted Transformation:
 [[-5.76600617e-01  7.42965817e-01  3.39902227e-01  3.21665218e+03]
 [ 7.60254369e-01  3.35528968e-01  5.56267566e-01  1.20546238e+03]
 [ 2.99240743e-01  5.79156375e-01 -7.58309219e-01  4.33474833e+03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Alignment Error: 2.832500664323631e-11


### 1.2 Implement ICP algorithm with unknown correspondences. (5 Points)

Your task is to write a function that implements ICP and takes two point clouds as input wherein the correspondances are unknown. Visualize the pointclouds and plot their individual coordinate frames as you perform ICP over them. Compute the alignment error in each iteration. 

Refer to Shubodh's notes to compute correspondences: https://saishubodh.notion.site/Mobile-Robotics-Navigating-from-Theory-to-Application-0b65a9c20edd4081978f4ffad917febb?p=a25686ce1a11409d838d47bcac43ab4b&pm=s#bb9aaf2e316b4db3b399df1742f0444c


In [21]:
def icp_unknown_correspondences(P, Q, num_iterations=100):
    """ Given two transformation matrices, transforms and shuffles toothless.ply
    and performs alignment on the transformed point clouds. """

    # Read the point cloud
    pcd = o3d.io.read_point_cloud("data/toothless.ply")
    X = np.asarray(pcd.points)

    # Apply the transformation
    X_P = apply_transformation(X, P)
    X_Q = apply_transformation(X, Q)

    # Shuffle the second point cloud for unknown correspondences
    np.random.shuffle(X_Q)

    # Visualize the point clouds
    vis = o3d.visualization.Visualizer()
    vis.create_window()
    PC_X_P = o3d.geometry.PointCloud()
    PC_X_Q = o3d.geometry.PointCloud()
    PC_X_P.points = o3d.utility.Vector3dVector(X_P)
    PC_X_Q.points = o3d.utility.Vector3dVector(X_Q)
    vis.add_geometry(PC_X_P)
    vis.add_geometry(PC_X_Q)
    vis.poll_events()
    vis.update_renderer()

    # Initialize the error and predicted transformation
    error = 0
    T_pred = np.eye(4)

    for iteration in range(num_iterations):
        # Subsample point clouds
        X_P_sampled = X_P[np.random.choice(X_P.shape[0], 1000, replace=False)]
        X_Q_sampled = X_Q[np.random.choice(X_Q.shape[0], 5000, replace=False)]

        # Determine corresponding pairs and align the sampled point cloud
        distance, indices = get_nearest_neighbors(X_P_sampled, X_Q_sampled)
        X_Q_sampled = X_Q_sampled[indices]

        # Predict the applied transformation
        T_pred_current = procrustes_alignment(X_P_sampled, X_Q_sampled)
        T_pred = T_pred_current @ T_pred

        # Apply the predicted transformation
        X_P = apply_transformation(X_P, T_pred_current)
        X_P_sampled = apply_transformation(X_P_sampled, T_pred_current)

        # Update the visualization
        PC_X_P.points = o3d.utility.Vector3dVector(X_P)
        vis.update_geometry(PC_X_P)
        vis.poll_events()
        vis.update_renderer()

        # Compute RMSE between the two subsampled point clouds
        error = np.sqrt(np.mean((X_P_sampled - X_Q_sampled) ** 2))
        print(f'Error (iteration {iteration}): {error:.4f}')

    # Terminate visualization
    vis.run()
    vis.destroy_window()

    return T_pred, error

def get_nearest_neighbors(P, Q):
    dist = np.linalg.norm(P[:, np.newaxis] - Q, axis=2)
    return np.min(dist, axis=1), np.argmin(dist, axis=1)

P = getTransform()
Q = getTransform()
T_pred, error = icp_unknown_correspondences(P, Q)

print('Original Transformation:\n', Q)
print('Predicted Transformation:\n', T_pred)
print('Alignment Error:', error)

Error (iteration 0): 39.8568
Error (iteration 1): 30.7565
Error (iteration 2): 22.7827
Error (iteration 3): 19.0444
Error (iteration 4): 18.6785
Error (iteration 5): 17.9875
Error (iteration 6): 15.4331
Error (iteration 7): 16.3381
Error (iteration 8): 17.8410
Error (iteration 9): 15.4440
Error (iteration 10): 16.0850
Error (iteration 11): 14.4375
Error (iteration 12): 13.6350
Error (iteration 13): 14.6319
Error (iteration 14): 13.8031
Error (iteration 15): 15.0569
Error (iteration 16): 13.6038
Error (iteration 17): 15.2525
Error (iteration 18): 14.0732
Error (iteration 19): 13.6789
Error (iteration 20): 13.6911
Error (iteration 21): 13.3054
Error (iteration 22): 13.0538
Error (iteration 23): 14.9965
Error (iteration 24): 14.0090
Error (iteration 25): 13.4618
Error (iteration 26): 13.0616
Error (iteration 27): 13.6686
Error (iteration 28): 13.7932
Error (iteration 29): 13.4190
Error (iteration 30): 13.2500
Error (iteration 31): 13.1303
Error (iteration 32): 12.7872
Error (iteration 33)

imDefLkup.c,419: The application disposed a key event with 243 serial.


Error (iteration 74): 13.9956
Error (iteration 75): 13.2354
Error (iteration 76): 13.6306
Error (iteration 77): 14.8691
Error (iteration 78): 13.5894
Error (iteration 79): 13.2454
Error (iteration 80): 13.0204
Error (iteration 81): 13.4585
Error (iteration 82): 13.4737
Error (iteration 83): 13.5682
Error (iteration 84): 13.9847
Error (iteration 85): 14.1159
Error (iteration 86): 14.2394
Error (iteration 87): 13.7116
Error (iteration 88): 12.6581
Error (iteration 89): 13.0712
Error (iteration 90): 13.1473
Error (iteration 91): 13.9199
Error (iteration 92): 13.7482
Error (iteration 93): 12.5802
Error (iteration 94): 13.2218
Error (iteration 95): 14.8380
Error (iteration 96): 13.2062
Error (iteration 97): 12.5720
Error (iteration 98): 14.7747
Error (iteration 99): 15.4270
Original Transformation:
 [[ 1.01688227e-01 -4.38081414e-01 -8.93165259e-01  4.93242590e+03]
 [ 1.37746845e-01 -8.82970497e-01  4.48763756e-01  2.74485769e+03]
 [-9.85233633e-01 -1.68664687e-01 -2.94433721e-02  2.2382010

## SECTION 2: ICP with Lie Groups

### 2.1 Predict the Transformation matrix between 2 point clouds with known correspondences (15 Points)
Perform the same task as 1.1 using Lie Group Optimization from scratch to predict the transformation between the 2 point clouds.

Refer: https://saishubodh.notion.site/Mobile-Robotics-Navigating-from-Theory-to-Application-0b65a9c20edd4081978f4ffad917febb?p=ee55fe5689794693910ab7861bef067b&pm=s#7b82d84766a84b63b91d859579e4886b


In [ ]:
T = getTransform()
def icp_with_lie(T):
    #### YOUR CODE HERE ####
    pass

## SECTION 3: Pose Graph Optimization with G2O
### Objective (5 Points)
A robot is travelling in a oval trajectory. It is equipped with wheel odometry for odometry information and RGBD sensors for loop closure information. Due to noise in wheel odometry it generates a noisy estimate of the trajectory. Our task is to use loop closure pairs to correct the drift.

We pose this problem as a pose graph optimization problem. In our graph, poses are the vertices and constraints are the edges. 

References:

1.) Class notes: https://saishubodh.notion.site/G2O-Edge-Types-d9f9ff63c77c4ceeb84b1e49085004e3

2.) Cyrill Stachniss lecture: https://www.youtube.com/watch?v=uHbRKvD8TWg 

### Given: 
In practical scenarios, we'd obtain the following from our sensors after some post-processing:

1. Initial position
2. Odometry Contraints/Edges: This "edge" information tells us relative transformation between two nodes. These two nodes are consecutive in the case of Odometry but not in the case of Loop Closure (next point).
3. Loop Closure Contraints/Edges: Remember that while optimizing, you have another kind of "anchor" edge as you've seen in 1. solved example.

You have been given a text file named `edges.txt` (in `data/`) which has all the above 3 and it follows G2O's format (as explained in class, [link here](https://saishubodh.notion.site/G2O-Edge-Types-d9f9ff63c77c4ceeb84b1e49085004e3) ). The ground truth is `gt.txt`.

Install g2o as mentioned in `g2o.ipynb` and optimise `edges.txt`

In [ ]:
### YOUR CODE HERE ###

### Evo (10 Points)
We need a measure of how good the trajectory is. The error/loss used earlier doesn't tell us much about how the trajectory differs from the ground truth. Here, we try to do just this - compute error metrics. Rather than computing these from scratch, we will just Evo - https://github.com/MichaelGrupp/evo/.

Look at the absolute pose error (APE) and relative pose error (RPE). What do they capture and how are they calculated (descriptive answer)? How do these metrics differ in methodology? Can we determine if the error is more along the x/y axis?

Answer the above questions and report errors for the obtained trajectory.

In [ ]:
### YOUR CODE HERE ###